# Atividade 2.1 - Zero reais de funções reais

**Curso:** Bacharelado em Ciência da Computação  
**Disciplina:** Matemática Computacional  
**Professor:** Claudomiro de Souza de Sales Junior
**Aluno:** Alessandro Reali Lopes Silva

---

## Esse Nootebook tem como objetivo apresentar a resolução da Atividade 2.1 - Zero reais de funções reais.
1. Zero reais de funções reais
2. Estudo Especial de Equações Polinomiais

In [31]:
from typing import Callable, Dict, Tuple, List, Any, Union
from IPython.display import display, Markdown
import math
import time
from dataclasses import dataclass, asdict

# Estrutura para os dados de CADA iteração (Histórico)
@dataclass
class Iteracao:
    iteracao: int
    x: float          # x_meio ou aproximação atual
    fx: float         # valor da função
    erro: float       # b - a ou |x_novo - x_velho|
    tempo: float      # ms
    operacoes: int
    decisoes: int

# Estrutura para o Resumo Final (Comparação)
@dataclass
class Resultado:
    metodo: str
    raiz: float
    f_raiz: float
    erro_final: float
    iteracoes_totais: int
    tempo_total: float
    ops_totais: int
    decisoes_totais: int

# --- FUNÇÃO AUXILIAR DE FORMATAÇÃO (CORRIGIDA) ---
def gerar_tabela_markdown(dados: List[Union[Iteracao, Resultado, Dict[str, Any]]]) -> None:
    """
    Gera uma tabela formatada em Markdown a partir de uma lista de Dataclasses OU Dicionários.
    Limita a exibição a 10 linhas máximo: 1 cabeçalho + 4 primeiras + 1 pontilhado + 4 últimas.
    """
    if not dados:
        display(Markdown("**Nenhum dado disponível para exibir.**"))
        return
    
    # Converte dataclasses para dicionários, se necessário
    if hasattr(dados[0], '__dataclass_fields__'):
        dados_processados = [asdict(d) for d in dados]
    else:
        dados_processados = dados # Já são dicionários

    # Pega as chaves (colunas) do primeiro item
    chaves = list(dados_processados[0].keys())
    
    # Cria o cabeçalho
    md = f"| {' | '.join(chaves)} |\n"
    md += f"| {' | '.join([':---:'] * len(chaves))} |\n" # Centraliza
    
    # Determinar quais linhas exibir
    num_dados = len(dados_processados)
    if num_dados > 10:
        # Limitar a 10 linhas: 4 primeiras + 1 pontilhado + 4 últimas
        linhas_a_exibir = dados_processados[:4] + ['...'] + dados_processados[-4:]
    else:
        linhas_a_exibir = dados_processados
    
    # Preenche as linhas
    for linha in linhas_a_exibir:
        if linha == '...':
            # Linha de pontilhado
            md += f"| {' | '.join(['...'] * len(chaves))} |\n"
        else:
            valores_formatados = []
            for k in chaves:
                val = linha[k]
                
                # Formatação inteligente
                if isinstance(val, float):
                    # Critério para notação científica: muito pequeno ou muito grande
                    if (abs(val) < 1e-4 and val != 0) or abs(val) > 1e6:
                        valores_formatados.append(f"{val:.4e}")
                    else:
                        valores_formatados.append(f"{val:.6f}")
                else:
                    valores_formatados.append(str(val))
                    
            md += f"| {' | '.join(valores_formatados)} |\n"
    
    display(Markdown(md))

In [32]:
def metodo_bissecao(
    f: Callable[[float], float],
    x_min: float,
    x_max: float,
    epsilon: float,
) -> Tuple[Resultado, List[Iteracao]]:
    """
    Implementa o método da bisseção com rastreamento detalhado de esforço computacional.
    """
    # --- Inicialização Global ---
    operacoes = 0
    decisoes = 0
    chamadas_f = 0

    # Validação Inicial
    y_min = f(x_min)
    y_max = f(x_max)
    chamadas_f += 2
    
    operacoes += 1 # Multiplicação
    decisoes += 1  # If
    if y_min * y_max >= 0:
        raise ValueError("f(x_min) e f(x_max) devem ter sinais opostos.")
    
    historico: List[Iteracao] = []
    iteracoes = 0
    
    erro = (x_max - x_min)
    operacoes += 1 # Subtração

    tempo_inicial = time.perf_counter()

    while erro > epsilon:
        iteracoes += 1  # Conta a iteração
        tempo_iteracao_inicial = time.perf_counter()
        
        # Reinicia contadores locais da iteração
        decisoes_iteracao = 1 # Já conta o check do while
        operacoes_iteracao = 0
        
        x_meio = (x_min + x_max) / 2
        operacoes_iteracao += 2 # Soma e divisão

        y_meio = f(x_meio)
        chamadas_f += 1 # Global, pois conta no total
        
        # --- Lógica de Atualização do Intervalo ---
        decisoes_iteracao += 1 # If y_meio == 0
        raiz_exata_encontrada = False
        
        if y_meio == 0:
            raiz_exata_encontrada = True
        else:
            operacoes_iteracao += 1 # Multiplicação teste sinal
            decisoes_iteracao += 1  # If/Else
            
            if y_min * y_meio < 0:
                x_max = x_meio
            else:
                x_min = x_meio
                y_min = y_meio

        # Atualiza o erro para a tabela
        erro = (x_max - x_min)
        operacoes_iteracao += 1
        
        # Atualiza acumuladores globais com o esforço desta rodada
        operacoes += operacoes_iteracao
        decisoes += decisoes_iteracao
        
        # --- Salvamento do Histórico ---
        tempo_passado = (time.perf_counter() - tempo_iteracao_inicial) * 1000
        
        historico.append(Iteracao(
            iteracao=iteracoes,
            x=x_meio,
            fx=y_meio,
            erro=erro,
            tempo=tempo_passado,
            operacoes=operacoes_iteracao,
            decisoes=decisoes_iteracao,
        ))
        
        # Se achou a raiz exata, finaliza o loop
        if raiz_exata_encontrada:
            break

    tempo_final = time.perf_counter()
    tempo_total = (tempo_final - tempo_inicial) * 1000 

    # --- Criação do Resultado Final ---
    resultado = Resultado(
        metodo="Bisseção",
        raiz=x_meio,
        f_raiz=y_meio,
        erro_final=erro,
        iteracoes_totais=iteracoes,
        tempo_total=tempo_total,
        ops_totais=operacoes,
        decisoes_totais=decisoes,
    )
    
    return resultado, historico


def metodo_falsa_posicao(
    f: Callable[[float], float],
    x_min: float,
    x_max: float,
    epsilon: float,
) -> Tuple[Resultado, List[Iteracao]]:
    """
    Implementa o Método da Falsa Posição (Regula Falsi).
    Similar à bisseção, mas usa a média ponderada (pesos) pelos valores da função.
    """
    # --- Inicialização Global ---
    operacoes = 0
    decisoes = 0
    chamadas_f = 0

    # 1. Validação e Setup Inicial
    y_min = f(x_min)
    y_max = f(x_max)
    chamadas_f += 2
    
    operacoes += 1 # Multiplicação teste sinal
    decisoes += 1  # If teste sinal
    
    if y_min * y_max >= 0:
        raise ValueError("f(x_min) e f(x_max) devem ter sinais opostos.")
    
    historico: List[Iteracao] = []
    iteracoes = 0
    
    x_antigo = x_min # Inicializa com qualquer valor dentro do intervalo
    erro = float('inf')

    tempo_inicial = time.perf_counter()

    while erro > epsilon:
        tempo_iteracao_inicial = time.perf_counter()
        
        decisoes_iteracao = 1 # Check do while
        operacoes_iteracao = 0
        iteracoes += 1

        # --- A Grande Diferença: A Fórmula da Posição ---
        # x = (x_min * f(x_max) - x_max * f(x_min)) / (f(x_max) - f(x_min))
        numerador = (x_min * y_max) - (x_max * y_min)
        denominador = (y_max - y_min)
        
        # Proteção contra divisão por zero (raro, mas possível se y_max == y_min)
        decisoes_iteracao += 1
        if denominador == 0:
            raise ValueError("Divisão por zero na fórmula da Falsa Posição.")
        
        x_falso = numerador / denominador
        
        # Contagem da fórmula: 2 mult, 2 sub, 1 div
        operacoes_iteracao += 5 

        # Avaliação da Função no novo ponto
        y_falso = f(x_falso)
        chamadas_f += 1 # Global
        
        # --- Lógica de Atualização (Igual Bisseção) ---
        decisoes_iteracao += 1 # If y_falso == 0
        raiz_exata = False

        if y_falso == 0:
            raiz_exata = True
        else:
            operacoes_iteracao += 1 # Multiplicação teste
            decisoes_iteracao += 1  # If/Else
            
            if y_min * y_falso < 0:
                x_max = x_falso
                y_max = y_falso
            else:
                x_min = x_falso
                y_min = y_falso

        # Atualiza erro
        erro = abs(x_falso - x_antigo)
        operacoes_iteracao += 1 # Subtração

        x_antigo = x_falso     

        # Atualiza globais
        operacoes += operacoes_iteracao
        decisoes += decisoes_iteracao

        # Salva Histórico
        tempo_passado = (time.perf_counter() - tempo_iteracao_inicial) * 1000
        
        historico.append(Iteracao(
            iteracao=iteracoes,
            x=x_falso,
            fx=y_falso,
            erro=erro,
            tempo=tempo_passado,
            operacoes=operacoes_iteracao,
            decisoes=decisoes_iteracao,
        ))
        
        if raiz_exata:
            break

    tempo_final = time.perf_counter()
    tempo_total = (tempo_final - tempo_inicial) * 1000

    resultado = Resultado(
        metodo="Falsa Posição",
        raiz=x_falso,
        f_raiz=y_falso,
        erro_final=erro,
        iteracoes_totais=iteracoes,
        tempo_total=tempo_total,
        ops_totais=operacoes,
        decisoes_totais=decisoes,
    )
    
    return resultado, historico
    

def metodo_ponto_fixo(
    f: Callable[[float], float],
    phi: Callable[[float], float],
    x0: float,
    epsilon: float,
    max_iter: int = 100
) -> Tuple[Resultado, List[Iteracao]]:
    """
    Implementa o Método do Ponto Fixo (MPF).
    Requer a função de iteração 'phi' tal que x = phi(x).
    """
    # --- Inicialização Global ---
    operacoes = 0
    decisoes = 0
    chamadas_f = 0 # Contaremos chamadas para f() e phi()

    historico: List[Iteracao] = []
    iteracoes = 0
    x_atual = x0
    erro = float('inf')

    tempo_inicial = time.perf_counter()

    while erro > epsilon and iteracoes < max_iter:
        tempo_iteracao_inicial = time.perf_counter()
        
        decisoes_iteracao = 1 # Check do while
        operacoes_iteracao = 0
        iteracoes += 1

        # --- O Passo do MPF ---
        # A mágica acontece aqui: x(k+1) = phi(x(k))
        x_novo = phi(x_atual)
        chamadas_f += 1 

        # Avaliamos f(x) apenas para ver quão longe do zero estamos
        f_x_novo = f(x_novo)
        chamadas_f += 1

        # Cálculo do Erro (Critério de parada: distância entre x_novo e x_atual)
        erro = abs(x_novo - x_atual)
        operacoes_iteracao += 1 # Subtração

        # Atualiza globais
        operacoes += operacoes_iteracao
        decisoes += decisoes_iteracao

        # Salva Histórico
        tempo_passado = (time.perf_counter() - tempo_iteracao_inicial) * 1000
        
        historico.append(Iteracao(
            iteracao=iteracoes,
            x=x_novo,
            fx=f_x_novo,
            erro=erro,
            tempo=tempo_passado,
            operacoes=operacoes_iteracao,
            decisoes=decisoes_iteracao,
        ))

        # Atualiza para a próxima volta
        x_atual = x_novo
        
        # O MPF diverge muito fácil, então é bom ter um check de segurança
        decisoes += 1
        if erro > 1e6: # Se o erro explodiu para infinito
            print("Aviso: MPF Divergiu!")
            break

    tempo_final = time.perf_counter()
    tempo_total = (tempo_final - tempo_inicial) * 1000

    resultado = Resultado(
        metodo="Ponto Fixo",
        raiz=x_atual,
        f_raiz=f(x_atual),
        erro_final=erro,
        iteracoes_totais=iteracoes,
        tempo_total=tempo_total,
        ops_totais=operacoes,
        decisoes_totais=decisoes,
    )
    
    return resultado, historico


def metodo_newton(
    f: Callable[[float], float],
    df: Callable[[float], float],
    x0: float,
    epsilon: float,
    max_iter: int = 100
) -> Tuple[Resultado, List[Iteracao]]:
    """
    Implementa o método de Newton-Raphson com rastreamento detalhado de esforço computacional.
    """
    # --- Inicialização Global ---
    operacoes = 0
    decisoes = 0
    chamadas_f = 0 # Contaremos f(x) e f'(x) como chamadas

    historico: List[Iteracao] = []
    iteracoes = 0
    x_n = x0
    erro = float('inf')
    
    tempo_inicial = time.perf_counter()

    # Loop principal
    while erro > epsilon and iteracoes < max_iter:
        tempo_iteracao_inicial = time.perf_counter()
        
        # Reinicia contadores locais
        decisoes_iteracao = 1 # Check do while
        operacoes_iteracao = 0
        iteracoes += 1

        # 1. Avaliações (Função e Derivada)
        f_xn = f(x_n)
        df_xn = df(x_n)
        chamadas_f += 2 # Uma para f, uma para df
        
        # 2. Verificação Crítica (Derivada Zero)
        decisoes_iteracao += 1
        if df_xn == 0:
            # Salvamos o que temos até agora antes de falhar
            raise ValueError(f"Derivada zero encontrada na iteração {iteracoes}. O método falhou.")

        # 3. Passo de Newton: x(n+1) = x(n) - f(x)/f'(x)
        x_n1 = x_n - (f_xn / df_xn)
        operacoes_iteracao += 2 # Uma divisão, uma subtração

        # 4. Cálculo do Erro
        erro = abs(x_n1 - x_n)
        operacoes_iteracao += 1 # Subtração

        # Atualiza acumuladores globais
        operacoes += operacoes_iteracao
        decisoes += decisoes_iteracao

        # --- Salvamento do Histórico ---
        tempo_passado = (time.perf_counter() - tempo_iteracao_inicial) * 1000
        
        historico.append(Iteracao(
            iteracao=iteracoes,
            x=x_n,       # Mostramos o X usado nesta iteração
            fx=f_xn,     # O valor da função neste X
            erro=erro,   # O erro gerado para o próximo passo
            tempo=tempo_passado,
            operacoes=operacoes_iteracao,
            decisoes=decisoes_iteracao,
        ))

        # Atualiza x para a próxima rodada
        x_n = x_n1
        
        # Critério de parada extra: Raiz exata
        decisoes += 1 # Check do if
        if f_xn == 0:
            break

    tempo_final = time.perf_counter()
    tempo_total = (tempo_final - tempo_inicial) * 1000

    # --- Criação do Resultado Final ---
    # Nota: x_n agora contém o valor da última iteração (x_n1 da volta anterior)
    f_final = f(x_n)
    chamadas_f += 1 
    
    resultado = Resultado(
        metodo="Newton",
        raiz=x_n,
        f_raiz=f_final, 
        erro_final=erro,
        iteracoes_totais=iteracoes,
        tempo_total=tempo_total,
        ops_totais=operacoes,
        decisoes_totais=decisoes,
    )

    return resultado, historico

def metodo_secante(
    f: Callable[[float], float],
    x0: float,
    x1: float,
    epsilon: float,
    max_iter: int = 100
) -> Tuple[Resultado, List[Iteracao]]:
    """
    Implementa o método da secante com rastreamento detalhado de esforço computacional.
    """
    # --- Inicialização Global ---
    operacoes = 0
    decisoes = 0
    chamadas_f = 0 

    # 1. Pré-cálculo
    f_x0 = f(x0)
    f_x1 = f(x1)
    chamadas_f += 2
    
    historico: List[Iteracao] = []
    iteracoes = 0
    erro = float('inf')
    
    tempo_inicial = time.perf_counter()

    # Loop principal
    while erro > epsilon and iteracoes < max_iter:
        tempo_iteracao_inicial = time.perf_counter()
        
        # Reinicia contadores locais
        decisoes_iteracao = 1 # Check do while
        operacoes_iteracao = 0
        iteracoes += 1

        # 2. Verificação Crítica (Denominador Zero)
        # O denominador é (f(x1) - f(x0))
        denominador = f_x1 - f_x0
        operacoes_iteracao += 1 # Subtração
        
        decisoes_iteracao += 1
        if denominador == 0:
            raise ValueError(f"Divisão por zero na iteração {iteracoes}. f(x1) é igual a f(x0).")

        # 3. Passo da Secante: x2 = x1 - (f(x1) * (x1 - x0) / denominador)
        x2 = x1 - (f_x1 * (x1 - x0) / denominador)
        operacoes_iteracao += 4 # 1 sub (x1-x0), 1 mult, 1 div, 1 sub final

        # 4. Cálculo do Erro
        erro = abs(x2 - x1)
        operacoes_iteracao += 1 # Subtração/Abs

        f_x2 = f(x2)
        chamadas_f += 1

        # Atualiza acumuladores globais
        operacoes += operacoes_iteracao
        decisoes += decisoes_iteracao

        # --- Salvamento do Histórico ---
        tempo_passado = (time.perf_counter() - tempo_iteracao_inicial) * 1000
        
        historico.append(Iteracao(
            iteracao=iteracoes,
            x=x2,
            fx=f_x2,
            erro=erro,
            tempo=tempo_passado,
            operacoes=operacoes_iteracao,
            decisoes=decisoes_iteracao,
        ))
        
        # Critério de sorte (Raiz exata)
        decisoes += 1
        if f_x2 == 0:
            x1 = x2 # Para o resultado final apontar certo
            f_x1 = f_x2
            break

        # Prepara a próxima iteração (Dança das Cadeiras)
        # x0 assume o lugar de x1, e x1 assume o lugar de x2
        x0 = x1
        f_x0 = f_x1
        
        x1 = x2
        f_x1 = f_x2

    tempo_final = time.perf_counter()
    tempo_total = (tempo_final - tempo_inicial) * 1000 

    resultado = Resultado(
        metodo="Secante",
        raiz=x1,
        f_raiz=f_x1,
        erro_final=erro,
        iteracoes_totais=iteracoes,
        tempo_total=tempo_total,
        ops_totais=operacoes,
        decisoes_totais=decisoes,
    )

    return resultado, historico

### Zero reais de funções reais


In [33]:
# --- EXEMPLO 18 ---
# f(x) = e^(-x^2) - cos(x)
# Raízes prováveis: x=0 e outra próxima de 1.44

def f18(x: float) -> float:
    """Função exemplo 18: f(x) = e^(-x^2) - cos(x)"""
    return math.exp(-(x**2)) - math.cos(x)

def df18(x: float) -> float:
    """
    Derivada f'(x):
    d/dx(e^(-x^2)) = -2x * e^(-x^2)
    d/dx(-cos(x))  = sin(x)
    """
    return -2 * x * math.exp(-(x**2)) + math.sin(x)

def phi_f18(x: float) -> float:
    """
    Função phi para o MPF.
    Isolando x em cos(x) = e^(-x^2) -> x = arccos(e^(-x^2))
    Nota: Converge para a raiz positiva se x0 for adequado.
    """
    try:
        val = math.exp(-(x**2))
        # Proteção matemática: arccos só aceita valor entre -1 e 1
        if val > 1:
            val = 1
        return math.acos(val)
    except ValueError:
        return x # Retorna o próprio x em caso de erro para não quebrar o loop

# --- EXEMPLO 19 ---
# f(x) = x^3 - x - 1
# Raiz provável: ~1.32

def f19(x: float) -> float:
    """Função exemplo 19: f(x) = x^3 - x - 1"""
    return x**3 - x - 1

def df19(x: float) -> float:
    """Derivada f'(x) = 3x^2 - 1"""
    return 3 * (x**2) - 1

def phi_f19(x: float) -> float:
    """
    Função phi para o MPF.
    Isolando x do termo cúbico: x^3 = x + 1 -> x = (x+1)^(1/3)
    Esta forma geralmente converge (derivada < 1 perto da raiz).
    """
    # math.pow funciona melhor com bases positivas.
    if x + 1 < 0:
        return -((- (x + 1))**(1/3)) # Tratamento para raiz cúbica de negativo
    return (x + 1)**(1/3)

# --- EXEMPLO 20 ---
# f(x) = 4sin(x) - e^x
# Raiz provável: ~0.37

def f20(x: float) -> float:
    """Função exemplo 20: f(x) = 4sin(x) - e^x"""
    return 4 * math.sin(x) - math.exp(x)

def df20(x: float) -> float:
    """Derivada f'(x) = 4cos(x) - e^x"""
    return 4 * math.cos(x) - math.exp(x)

def phi_f20(x: float) -> float:
    """
    Função phi para o MPF.
    Isolando x do seno: 4sin(x) = e^x -> sin(x) = e^x / 4 -> x = arcsin(e^x / 4)
    """
    try:
        val = math.exp(x) / 4
        # Proteção: arcsin só aceita entre -1 e 1
        if val > 1 or val < -1: 
            return x # Evita erro matemático mantendo o valor
        return math.asin(val)
    except ValueError:
        return x

# --- EXEMPLO 21 ---
# f(x) = x * ln(x) - 1
# Raiz provável: ~1.76

def f21(x: float) -> float:
    """Função exemplo 21: f(x) = x * ln(x) - 1"""
    # Proteção: log(x) só existe para x > 0
    if x <= 0:
        return -1e9 # Valor dummy para indicar erro
    return x * math.log(x) - 1

def df21(x: float) -> float:
    """
    Derivada f'(x): Regra do Produto
    1 * ln(x) + x * (1/x) = ln(x) + 1
    """
    if x <= 0:
        return 1e9
    return math.log(x) + 1

def phi_f21(x: float) -> float:
    """
    Função phi para o MPF.
    Isolando x: ln(x) = 1/x -> x = e^(1/x)
    """
    if x == 0:
        return x
    return math.exp(1/x)

# --- EXEMPLO 22 ---
# f(x) = x^3 - 3.5x^2 + 4x - 1.5
# Polinômio de grau 3

def f22(x: float) -> float:
    """Função exemplo 22: f(x) = x^3 - 3.5x^2 + 4x - 1.5"""
    return x**3 - 3.5*(x**2) + 4*x - 1.5

def df22(x: float) -> float:
    """Derivada f'(x) = 3x^2 - 7x + 4"""
    return 3*(x**2) - 7*x + 4

def phi_f22(x: float) -> float:
    """
    Função phi para o MPF.
    Isolando o termo linear 4x:
    4x = -x^3 + 3.5x^2 + 1.5
    x = (-x^3 + 3.5x^2 + 1.5) / 4
    """
    return (- (x**3) + 3.5*(x**2) + 1.5) / 4

In [34]:
# --- FUNÇÃO ORQUESTRADORA (RESOLVE UM EXEMPLO COMPLETO) ---
def executar_analise_completa(
    numero_exemplo: int,
    f: Callable[[float], float],
    df: Callable[[float], float],
    phi: Callable[[float], float],
    x_min: float,
    x_max: float,
    epsilon: float = 1e-4
):
    """
    Executa os 5 métodos numéricos para uma dada função e exibe:
    1. Tabela detalhada da Bisseção (Requisito 'Reproduzir Tabelas')
    2. Tabela comparativa final (Ranking de eficiência)
    """
    
    titulo = f"### Resolução do Exemplo {numero_exemplo}"
    display(Markdown(f"{titulo}"))
    display(Markdown(f"**Intervalo:** [{x_min}, {x_max}] | **Precisão:** {epsilon}"))
    
    lista_comparativa: List[Resultado] = []
    
    # Definição de chutes iniciais baseados no intervalo
    x0_medio = (x_min + x_max) / 2  # Para Newton e MPF
    
    # ---------------------------------------------------------
    # 1. MÉTODO DA BISSEÇÃO (O Clássico)
    # ---------------------------------------------------------
    try:
        res_bi, hist_bi = metodo_bissecao(f, x_min, x_max, epsilon)
        lista_comparativa.append(res_bi)
        
        display(Markdown("#### 1. Detalhamento (Bisseção)"))
        display(Markdown("Tabela passo a passo conforme solicitado:"))
        gerar_tabela_markdown(hist_bi) # Mostra o histórico completo
    except Exception as e:
        print(f"Erro na Bisseção: {e}")

    # ---------------------------------------------------------
    # 2. MÉTODO DA FALSA POSIÇÃO
    # ---------------------------------------------------------
    try:
        res_fp, hist_fp = metodo_falsa_posicao(f, x_min, x_max, epsilon)
        lista_comparativa.append(res_fp)
        display(Markdown("#### 2. Detalhamento (Falsa Posição)"))
        display(Markdown("Tabela passo a passo conforme solicitado:"))
        gerar_tabela_markdown(hist_fp) # Mostra o histórico completo
    except Exception as e:
        print(f"Erro na Falsa Posição: {e}")

    # ---------------------------------------------------------
    # 3. MÉTODO DO PONTO FIXO (MPF)
    # ---------------------------------------------------------
    try:
        # Usa o ponto médio como chute inicial
        res_mpf, hist_mpf = metodo_ponto_fixo(f, phi, x0_medio, epsilon)
        lista_comparativa.append(res_mpf)
        display(Markdown("#### 3. Detalhamento (Ponto Fixo)"))
        display(Markdown("Tabela passo a passo conforme solicitado:"))
        gerar_tabela_markdown(hist_mpf) # Mostra o histórico completo
    except Exception as e:
        print(f"Erro no MPF: {e}")

    # ---------------------------------------------------------
    # 4. MÉTODO DE NEWTON-RAPHSON
    # ---------------------------------------------------------
    try:
        res_newton, hist_newton = metodo_newton(f, df, x0_medio, epsilon)
        lista_comparativa.append(res_newton)
        display(Markdown("#### 4. Detalhamento (Newton-Raphson)"))
        display(Markdown("Tabela passo a passo conforme solicitado:"))
        gerar_tabela_markdown(hist_newton) # Mostra o histórico completo
    except Exception as e:
        print(f"Erro no Newton: {e}")

    # ---------------------------------------------------------
    # 5. MÉTODO DA SECANTE
    # ---------------------------------------------------------
    try:
        # Usa os extremos do intervalo como os dois primeiros pontos
        res_sec, hist_sec = metodo_secante(f, x_min, x_max, epsilon)
        lista_comparativa.append(res_sec)
        display(Markdown("#### 5. Detalhamento (Secante)"))
        display(Markdown("Tabela passo a passo conforme solicitado:"))
        gerar_tabela_markdown(hist_sec) # Mostra o histórico completo
    except Exception as e:
        print(f"Erro na Secante: {e}")

    # ---------------------------------------------------------
    # EXIBIÇÃO DO RANKING FINAL
    # ---------------------------------------------------------
    display(Markdown("#### Quadro Comparativo Final"))
    gerar_tabela_markdown(lista_comparativa)
    display(Markdown("---")) # Separador visual

In [35]:
executar_analise_completa(
    numero_exemplo=18,
    f=f18,
    df=df18,
    phi=phi_f18,
    x_min=1.0,
    x_max=2.0,
    epsilon=1e-4
)

### Resolução do Exemplo 18

**Intervalo:** [1.0, 2.0] | **Precisão:** 0.0001

#### 1. Detalhamento (Bisseção)

Tabela passo a passo conforme solicitado:

| iteracao | x | fx | erro | tempo | operacoes | decisoes |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| 1 | 1.500000 | 0.034662 | 0.500000 | 0.001700 | 4 | 3 |
| 2 | 1.250000 | -0.105711 | 0.250000 | 0.002200 | 4 | 3 |
| 3 | 1.375000 | -0.043570 | 0.125000 | 0.002000 | 4 | 3 |
| 4 | 1.437500 | -0.006262 | 0.062500 | 0.001300 | 4 | 3 |
| ... | ... | ... | ... | ... | ... | ... |
| 11 | 1.447754 | 0.000216 | 0.000488 | 0.000700 | 4 | 3 |
| 12 | 1.447510 | 6.0751e-05 | 0.000244 | 0.000700 | 4 | 3 |
| 13 | 1.447388 | -1.6906e-05 | 0.000122 | 0.000700 | 4 | 3 |
| 14 | 1.447449 | 2.1921e-05 | 6.1035e-05 | 0.000600 | 4 | 3 |


...

Tabela passo a passo conforme solicitado:

| iteracao | x | fx | erro | tempo | operacoes | decisoes |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| 1 | 1.284111 | -0.090521 | 0.715889 | 0.001600 | 6 | 2 |
| 2 | 1.407549 | -0.024619 | 0.123438 | 0.001800 | 6 | 2 |
| 3 | 1.453661 | 0.003991 | 0.046112 | 0.001000 | 6 | 2 |
| 4 | 1.447228 | -0.000119 | 0.006433 | 0.000700 | 6 | 2 |
| 5 | 1.447413 | -5.2422e-07 | 0.000186 | 0.000600 | 6 | 2 |
| 6 | 1.447414 | 6.9758e-11 | 8.2419e-07 | 0.000400 | 6 | 2 |


#### Quadro Comparativo Final

| metodo | raiz | f_raiz | erro_final | iteracoes_totais | tempo_total | ops_totais | decisoes_totais |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| Bisseção | 1.447449 | 2.1921e-05 | 6.1035e-05 | 14 | 0.039700 | 58 | 43 |
| Falsa Posição | 1.447403 | -6.9477e-06 | 4.6282e-05 | 7 | 0.015500 | 50 | 29 |
| Ponto Fixo | 1.447451 | 2.3482e-05 | 6.5923e-05 | 7 | 0.018800 | 7 | 14 |
| Newton | 1.447414 | 1.9567e-12 | 2.0757e-06 | 3 | 0.014400 | 9 | 9 |
| Secante | 1.447414 | 6.9758e-11 | 8.2419e-07 | 6 | 0.013500 | 36 | 18 |


---

In [36]:
executar_analise_completa(
    numero_exemplo=19,
    f=f19,
    df=df19,
    phi=phi_f19,
    x_min=1.0,
    x_max=2.0,
    epsilon=1e-6
)

### Resolução do Exemplo 19

**Intervalo:** [1.0, 2.0] | **Precisão:** 1e-06

#### 1. Detalhamento (Bisseção)

Tabela passo a passo conforme solicitado:

| iteracao | x | fx | erro | tempo | operacoes | decisoes |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| 1 | 1.500000 | 0.875000 | 0.500000 | 0.001200 | 4 | 3 |
| 2 | 1.250000 | -0.296875 | 0.250000 | 0.000800 | 4 | 3 |
| 3 | 1.375000 | 0.224609 | 0.125000 | 0.000700 | 4 | 3 |
| 4 | 1.312500 | -0.051514 | 0.062500 | 0.000600 | 4 | 3 |
| ... | ... | ... | ... | ... | ... | ... |
| 17 | 1.324715 | -1.4059e-05 | 7.6294e-06 | 0.000400 | 4 | 3 |
| 18 | 1.324718 | 2.2095e-06 | 3.8147e-06 | 0.000200 | 4 | 3 |
| 19 | 1.324717 | -5.9246e-06 | 1.9073e-06 | 0.000300 | 4 | 3 |
| 20 | 1.324718 | -1.8576e-06 | 9.5367e-07 | 0.000400 | 4 | 3 |


...

Tabela passo a passo conforme solicitado:

| iteracao | x | fx | erro | tempo | operacoes | decisoes |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| 1 | 1.166667 | -0.578704 | 0.833333 | 0.001800 | 6 | 2 |
| 2 | 1.253112 | -0.285363 | 0.086445 | 0.001000 | 6 | 2 |
| 3 | 1.337206 | 0.053881 | 0.084094 | 0.000800 | 6 | 2 |
| 4 | 1.323850 | -0.003698 | 0.013356 | 0.000700 | 6 | 2 |
| 5 | 1.324708 | -4.2734e-05 | 0.000858 | 0.000500 | 6 | 2 |
| 6 | 1.324718 | 3.4582e-08 | 1.0029e-05 | 0.001000 | 6 | 2 |
| 7 | 1.324718 | -3.2285e-13 | 8.1091e-09 | 0.000700 | 6 | 2 |


#### Quadro Comparativo Final

| metodo | raiz | f_raiz | erro_final | iteracoes_totais | tempo_total | ops_totais | decisoes_totais |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| Bisseção | 1.324718 | -1.8576e-06 | 9.5367e-07 | 20 | 0.023600 | 82 | 61 |
| Falsa Posição | 1.324717 | -1.9552e-06 | 6.2273e-07 | 16 | 0.021700 | 113 | 65 |
| Ponto Fixo | 1.324718 | 2.3346e-07 | 2.3346e-07 | 9 | 0.013700 | 9 | 18 |
| Newton | 1.324718 | 1.8652e-13 | 2.1675e-07 | 4 | 0.013700 | 12 | 12 |
| Secante | 1.324718 | -3.2285e-13 | 8.1091e-09 | 7 | 0.016100 | 42 | 21 |


---

In [37]:
executar_analise_completa(
    numero_exemplo=20,
    f=f20,
    df=df20,
    phi=phi_f20,
    x_min=0.0,
    x_max=1.0,
    epsilon=1e-5
)

### Resolução do Exemplo 20

**Intervalo:** [0.0, 1.0] | **Precisão:** 1e-05

#### 1. Detalhamento (Bisseção)

Tabela passo a passo conforme solicitado:

| iteracao | x | fx | erro | tempo | operacoes | decisoes |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| 1 | 0.500000 | 0.268981 | 0.500000 | 0.001000 | 4 | 3 |
| 2 | 0.250000 | -0.294410 | 0.250000 | 0.000700 | 4 | 3 |
| 3 | 0.375000 | 0.010099 | 0.125000 | 0.000500 | 4 | 3 |
| 4 | 0.312500 | -0.137084 | 0.062500 | 0.000400 | 4 | 3 |
| ... | ... | ... | ... | ... | ... | ... |
| 14 | 0.370544 | -3.1150e-05 | 6.1035e-05 | 0.000300 | 4 | 3 |
| 15 | 0.370575 | 3.8429e-05 | 3.0518e-05 | 0.000400 | 4 | 3 |
| 16 | 0.370560 | 3.6398e-06 | 1.5259e-05 | 0.000400 | 4 | 3 |
| 17 | 0.370552 | -1.3755e-05 | 7.6294e-06 | 0.000300 | 4 | 3 |


...

Tabela passo a passo conforme solicitado:

| iteracao | x | fx | erro | tempo | operacoes | decisoes |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| 1 | 0.606943 | 0.446622 | 0.393057 | 0.001500 | 6 | 2 |
| 2 | -0.266518 | -1.819540 | 0.873461 | 0.000900 | 6 | 2 |
| 3 | 0.434798 | 0.140258 | 0.701317 | 0.000400 | 6 | 2 |
| 4 | 0.384607 | 0.031742 | 0.050192 | 0.000600 | 6 | 2 |
| 5 | 0.369925 | -0.001443 | 0.014681 | 0.000400 | 6 | 2 |
| 6 | 0.370564 | 1.3094e-05 | 0.000639 | 0.000500 | 6 | 2 |
| 7 | 0.370558 | 5.2605e-09 | 5.7406e-06 | 0.000500 | 6 | 2 |


#### Quadro Comparativo Final

| metodo | raiz | f_raiz | erro_final | iteracoes_totais | tempo_total | ops_totais | decisoes_totais |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| Bisseção | 0.370552 | -1.3755e-05 | 7.6294e-06 | 17 | 0.020200 | 70 | 52 |
| Falsa Posição | 0.370559 | 1.6698e-06 | 3.9883e-06 | 8 | 0.014100 | 57 | 33 |
| Ponto Fixo | 0.370563 | 1.0216e-05 | 7.0527e-06 | 11 | 0.020200 | 11 | 22 |
| Newton | 0.370558 | -4.4409e-16 | 1.2209e-08 | 4 | 0.017100 | 12 | 12 |
| Secante | 0.370558 | 5.2605e-09 | 5.7406e-06 | 7 | 0.013000 | 42 | 21 |


---

In [38]:
executar_analise_completa(
    numero_exemplo=21,
    f=f21,
    df=df21,
    phi=phi_f21,
    x_min=2.0,
    x_max=3.0,
    epsilon=1e-7
)

### Resolução do Exemplo 21

**Intervalo:** [2.0, 3.0] | **Precisão:** 1e-07

Erro na Bisseção: f(x_min) e f(x_max) devem ter sinais opostos.
Erro na Falsa Posição: f(x_min) e f(x_max) devem ter sinais opostos.


#### 3. Detalhamento (Ponto Fixo)

Tabela passo a passo conforme solicitado:

...

Tabela passo a passo conforme solicitado:

| iteracao | x | fx | erro | tempo | operacoes | decisoes |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| 1 | 1.797703 | 0.054371 | 1.202297 | 0.001500 | 6 | 2 |
| 2 | 1.768539 | 0.008340 | 0.029164 | 0.001000 | 6 | 2 |
| 3 | 1.763256 | 5.1230e-05 | 0.005284 | 0.000600 | 6 | 2 |
| 4 | 1.763223 | 4.9187e-08 | 3.2659e-05 | 0.000800 | 6 | 2 |
| 5 | 1.763223 | 2.9110e-13 | 3.1386e-08 | 0.000600 | 6 | 2 |


#### Quadro Comparativo Final

| metodo | raiz | f_raiz | erro_final | iteracoes_totais | tempo_total | ops_totais | decisoes_totais |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| Ponto Fixo | 1.763223 | 3.4977e-08 | 6.1673e-08 | 30 | 0.034000 | 30 | 60 |
| Newton | 1.763223 | 1.9984e-15 | 8.6329e-08 | 4 | 0.017100 | 12 | 12 |
| Secante | 1.763223 | 2.9110e-13 | 3.1386e-08 | 5 | 0.012500 | 30 | 15 |


---

In [39]:
executar_analise_completa(
    numero_exemplo=22,
    f=f22,
    df=df22,
    phi=phi_f22,
    x_min=0.0,
    x_max=2.0,
    epsilon=1e-4
)

### Resolução do Exemplo 22

**Intervalo:** [0.0, 2.0] | **Precisão:** 0.0001

#### 1. Detalhamento (Bisseção)

Tabela passo a passo conforme solicitado:

| iteracao | x | fx | erro | tempo | operacoes | decisoes |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| 1 | 1.000000 | 0.000000 | 2.000000 | 0.001300 | 3 | 2 |


...

Tabela passo a passo conforme solicitado:

| iteracao | x | fx | erro | tempo | operacoes | decisoes |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| 1 | 1.500000 | 0.000000 | 0.500000 | 0.001400 | 6 | 2 |


#### Quadro Comparativo Final

| metodo | raiz | f_raiz | erro_final | iteracoes_totais | tempo_total | ops_totais | decisoes_totais |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| Bisseção | 1.000000 | 0.000000 | 2.000000 | 1 | 0.005600 | 5 | 3 |
| Falsa Posição | 1.500000 | 0.000000 | 1.500000 | 1 | 0.005500 | 7 | 4 |
| Ponto Fixo | 1.000000 | 0.000000 | 0.000000 | 1 | 0.008200 | 1 | 2 |
| Secante | 1.500000 | 0.000000 | 0.500000 | 1 | 0.004200 | 6 | 3 |


---

### Estudo Especial de Equações Polinomiais

In [40]:
def polinomio_B(x: float) -> float:
    """Função polinomial B(x) = x^5 - 3.7x^4 + 7.4x^3 - 10.8x^2 + 10.8x - 6.8"""
    return x**5 - 3.7*x**4 + 7.4*x**3 - 10.8*x**2 + 10.8*x - 6.8

def dpolinomio_B(x: float) -> float:
    """Derivada dB/dx = 5x^4 - 14.8x^3 + 22.2x^2 - 21.6x + 10.8"""
    return 5*x**4 - 14.8*x**3 + 22.2*x**2 - 21.6*x + 10.8

def phi_polinomio_B(x: float) -> float:
    """
    Função phi para o MPF.
    Isolando o termo linear 10.8x:
    10.8x = -x^5 + 3.7x^4 - 7.4x^3 + 10.8x^2 + 6.8
    x = (-x^5 + 3.7x^4 - 7.4x^3 + 10.8x^2 + 6.8) / 10.8
    """
    return (- (x**5) + 3.7*(x**4) - 7.4*(x**3) + 10.8*(x**2) + 6.8) / 10.8

executar_analise_completa(
    numero_exemplo=23,
    f=polinomio_B,
    df=dpolinomio_B,
    phi=phi_polinomio_B,
    x_min=1.0,
    x_max=2.0,
    epsilon=1e-5
)

### Resolução do Exemplo 23

**Intervalo:** [1.0, 2.0] | **Precisão:** 1e-05

#### 1. Detalhamento (Bisseção)

Tabela passo a passo conforme solicitado:

| iteracao | x | fx | erro | tempo | operacoes | decisoes |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| 1 | 1.500000 | -1.062500 | 0.500000 | 0.001900 | 4 | 3 |
| 2 | 1.750000 | 0.395508 | 0.250000 | 0.001400 | 4 | 3 |
| 3 | 1.625000 | -0.484003 | 0.125000 | 0.001700 | 4 | 3 |
| 4 | 1.687500 | -0.089237 | 0.062500 | 0.001500 | 4 | 3 |
| ... | ... | ... | ... | ... | ... | ... |
| 14 | 1.700012 | 8.8943e-05 | 6.1035e-05 | 0.000800 | 4 | 3 |
| 15 | 1.699982 | -0.000133 | 3.0518e-05 | 0.001100 | 4 | 3 |
| 16 | 1.699997 | -2.2235e-05 | 1.5259e-05 | 0.001000 | 4 | 3 |
| 17 | 1.700005 | 3.3353e-05 | 7.6294e-06 | 0.000400 | 4 | 3 |


...

Tabela passo a passo conforme solicitado:

| iteracao | x | fx | erro | tempo | operacoes | decisoes |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| 1 | 1.368421 | -1.458356 | 0.631579 | 0.002100 | 6 | 2 |
| 2 | 1.550509 | -0.857895 | 0.182088 | 0.001300 | 6 | 2 |
| 3 | 1.810663 | 0.968017 | 0.260154 | 0.001200 | 6 | 2 |
| 4 | 1.672741 | -0.189982 | 0.137922 | 0.001200 | 6 | 2 |
| 5 | 1.695369 | -0.033489 | 0.022628 | 0.000800 | 6 | 2 |
| 6 | 1.700211 | 0.001538 | 0.004842 | 0.000900 | 6 | 2 |
| 7 | 1.699998 | -1.1678e-05 | 0.000213 | 0.000500 | 6 | 2 |
| 8 | 1.700000 | -4.0282e-09 | 1.6022e-06 | 0.000600 | 6 | 2 |


#### Quadro Comparativo Final

| metodo | raiz | f_raiz | erro_final | iteracoes_totais | tempo_total | ops_totais | decisoes_totais |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| Bisseção | 1.700005 | 3.3353e-05 | 7.6294e-06 | 17 | 0.037400 | 70 | 52 |
| Falsa Posição | 1.699994 | -4.2297e-05 | 8.9729e-06 | 13 | 0.022100 | 92 | 53 |
| Ponto Fixo | 1.699998 | -1.5982e-05 | 4.5481e-06 | 11 | 0.034600 | 11 | 22 |
| Newton | 1.700000 | 4.3521e-14 | 5.7446e-08 | 5 | 0.014800 | 15 | 15 |
| Secante | 1.700000 | -4.0282e-09 | 1.6022e-06 | 8 | 0.015700 | 48 | 24 |


---